In [49]:
from torchvision.io import read_image
import torch
import matplotlib.pyplot as plt
from plyfile import PlyData, PlyElement
import numpy as np

In [105]:
depth_img=read_image("/root/BBSEA/images/lovers_depth.png")[0]
rgb_img=read_image("/root/BBSEA/images/lovers_rgb.png")

In [106]:
print(depth_img.shape)
print(rgb_img.shape)

torch.Size([219, 341])
torch.Size([4, 219, 341])


In [133]:
type(depth_img)

torch.Tensor

In [129]:
camera_factor, fx, fy, cx, cy = 4, 700, 700, 170.5, 109.5

height, width = depth_img.shape
x = torch.arange(width).view(1, -1).repeat(height, 1).float()
y = torch.arange(height).view(-1, 1).repeat(1, width).float()
x = (x - cx) / fx
y = (y - cy) / fy

z = (255-depth_img.float())/camera_factor  # 假设深度值越小表示越近

X = x * z
Y = y * z
Z = z

# 假设rgb_img是[3, height, width]，如果不是，请根据实际情况调整
R = rgb_img[0]
G = rgb_img[1]
B = rgb_img[2]

pointcloud = torch.stack((X, Y, Z, R, G, B), dim=-1)
# pointcloud = pointcloud.cpu().numpy()

In [130]:
print(Z.max(),Z.min())
print(Z)

tensor(63.7500) tensor(0.)
tensor([[55.7500, 57.0000, 57.0000,  ..., 60.7500, 60.7500, 60.7500],
        [54.7500, 42.5000, 43.5000,  ..., 62.2500, 62.2500, 62.2500],
        [54.7500, 42.5000, 44.0000,  ..., 62.2500, 62.2500, 62.2500],
        ...,
        [45.5000, 39.5000, 41.0000,  ..., 44.7500, 44.7500, 44.7500],
        [45.5000, 39.5000, 40.2500,  ..., 44.7500, 44.7500, 44.7500],
        [45.5000, 39.5000, 40.2500,  ..., 44.7500, 44.7500, 44.7500]])


In [131]:
pointcloud[105,170]

tensor([-6.9643e-03, -6.2679e-02,  9.7500e+00,  1.1000e+01,  2.8000e+01,
         4.2000e+01])

In [132]:
vertex = np.array([(pointcloud[i, j, 0], pointcloud[i, j, 1], pointcloud[i, j, 2],
                    pointcloud[i, j, 3], pointcloud[i, j, 4], pointcloud[i, j, 5])
                   for i in range(pointcloud.shape[0])
                   for j in range(pointcloud.shape[1])],
                  dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
                         ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

el = PlyElement.describe(vertex, 'vertex', comments=['vertices with color'])
PlyData([el], text=True).write('lovers_point_cloud.ply')